In [1]:
import json
import pickle
import random
import math
from time import time
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, concatenate, Dense, Input, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [2]:
class ImageCaptionEmbeddingWithConsensus:
    def __init__(self, image_name, caption, image_embedding, caption_embedding, consensus):
        self.image_name = image_name
        self.caption = caption
        self.image_embedding = image_embedding
        self.caption_embedding = caption_embedding
        self.consensus = consensus

In [12]:
# This cell is for data labeled with consensus. It includes all data extracted from the game. 
# Aug data can be used or ignored - it is 5000 new image/caption pairs with vsepp derived ratings. 
image_caption_objects = 'corrected_images_captions_with_embeddings.pkl'
mirrored_objects = 'mirrored_images_captions_with_embeddings.pkl'
aug_data = 'augmented_data_embeddings.pkl'
aug_data_mirrored = 'augmented_data_embeddings_mirrored.pkl'

# encoding = latin1: needs to be specified in order to unpickle numpy arrays from Python2 to Python3
with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
    
#with open(aug_data, 'rb') as f3:
 #   data_list_3 = pickle.load(f3, encoding='latin1')
  #  print(len(data_list_3))
    
#with open(aug_data_mirrored, 'rb') as f4:
 #   data_list_4 = pickle.load(f4, encoding='latin1')
  #  print(len(data_list_4))

12039
12039


In [13]:
data_list = data_list_1 + data_list_2 #+ data_list_3 + data_list_4
print(len(data_list))

24078


In [64]:
# This cell is here to run the model with human only ratings
# In these pkl files, the "consensus" field contains the human average of the human ratings
image_caption_objects = 'shorter_human_scores_only_embeddings.pkl'
mirrored_objects = 'shorter_human_scores_only_embeddings_mirrored.pkl'

with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
    

data_list = data_list_1 + data_list_2
print(len(data_list))

10631
10631
21262


In [86]:
# This cell is for running a trial with human only data, where there are 3 or more human ratings per sample
image_caption_objects = 'three_or_more_human_ratings.pkl'
mirrored_objects = 'three_or_more_human_ratings_mirrored.pkl'

with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
    

data_list = data_list_1 + data_list_2
print(len(data_list))

4292
4292
8584


In [106]:
# This cell is for running a trial with human only data, where there are 2 or more human ratings per sample
image_caption_objects = 'two_or_more_human_ratings.pkl'
mirrored_objects = 'two_or_more_human_ratings_mirrored.pkl'

with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
    

data_list = data_list_1 + data_list_2
print(len(data_list))

8565
8565
17130


In [27]:
# This cell is here to run the model with vsepp only ratings - Don't run this unless needed
# In these pkl files, the "consensus" field contains the vsepp bucketed rating (as pulled from game data)
# Since we are not relying on human ratings here, we have in total 24,078 samples (12,039 original + mirrored), without aug
# With augmented data, we have 34,078
image_caption_objects = 'vsepp_scores_only_embeddings.pkl'
mirrored_objects = 'vsepp_scores_only_embeddings_mirrored.pkl'
aug_data = 'augmented_data_embeddings.pkl'
aug_data_mirrored = 'augmented_data_embeddings_mirrored.pkl'

with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
  
with open(aug_data, 'rb') as f3:
    data_list_3 = pickle.load(f3, encoding='latin1')
    print(len(data_list_3))
    
with open(aug_data_mirrored, 'rb') as f4:
    data_list_4 = pickle.load(f4, encoding='latin1')
    print(len(data_list_4))

data_list = data_list_1 + data_list_2 + data_list_3 + data_list_4
print(len(data_list))

12039
12039
5000
5000
34078


In [3]:
# 10/17
# ----------------------------

# This cell is here to run the model with vsepp only ratings from all data
# In these pkl files, the "consensus" field contains the vsepp bucketed rating (as pulled from game data)
# Since we are not relying on human ratings here, we have in total 24,078 samples (12,039 original + mirrored), without aug
# Using 2 augmented datasets - the 5000 image/captions + mirrored from 9/13, and the 18,500 image/captions + mirrored from 10/17
# With all this data, we have 71,078 image/caption/ratings

image_caption_objects = 'vsepp_scores_only_embeddings.pkl'
mirrored_objects = 'vsepp_scores_only_embeddings_mirrored.pkl'
aug_data = 'augmented_data_embeddings.pkl'
aug_data_mirrored = 'augmented_data_embeddings_mirrored.pkl'
aug_data_2 = 'all_augment_data_10_17_embeddings.pkl'
aug_data_2_mirrored = 'all_augment_data_10_17_embeddings_mirrored.pkl'

with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
  
with open(aug_data, 'rb') as f3:
    data_list_3 = pickle.load(f3, encoding='latin1')
    print(len(data_list_3))
    
with open(aug_data_mirrored, 'rb') as f4:
    data_list_4 = pickle.load(f4, encoding='latin1')
    print(len(data_list_4))
    
with open(aug_data_2, 'rb') as f5:
    data_list_5 = pickle.load(f5, encoding='latin1')
    print(len(data_list_5))
    
with open(aug_data_2_mirrored, 'rb') as f6:
    data_list_6 = pickle.load(f6, encoding='latin1')
    print(len(data_list_6))

#data_list = data_list_1 + data_list_2 + data_list_3 + data_list_4 + data_list_5 + data_list_6
data_list = data_list_1 + data_list_3 + data_list_5
print(len(data_list))

12039
12039
5000
5000
18500
18500
35539


In [ ]:
print(data_list[1089].consensus)

In [4]:
# To concatenate, use numpy concatenate. axis = 1 results in a numpy array [[2048]], need to access [0]
#im_emb = data_list[0].image_embedding
#cap_emb = data_list[0].caption_embedding
#concat = np.concatenate((im_emb, cap_emb), axis = 1)

In [5]:
# Run this
# May need to exclude some data - especially data with no human ratings
# Note that it seems like with no human ratings, there is still consensus.
# And if there are no cosine similiarty data, there is still consensus ('Z0', 'Z1' don't always exist)
#list(filter(lambda x: x < 0, number_list))
cleaned_data_list = list(filter(lambda obj: float(obj.consensus) > 0, data_list))
random.shuffle(cleaned_data_list)
print(len(cleaned_data_list))

35539


In [119]:
def make_even_data(game_data_list):
    #print(len(game_data_list))
    evened_list = []
    list_of_consensus = []
    for obj in game_data_list:
        list_of_consensus.append(int(float(obj.consensus)))
    
    count_1 = list_of_consensus.count(1)
    count_2 = list_of_consensus.count(2)
    count_3 = list_of_consensus.count(3)
    count_4 = list_of_consensus.count(4)
    count_5 = list_of_consensus.count(5)
    
    #print(count_1, count_2, count_3, count_4, count_5)
    #print(count_1 + count_2 + count_3 + count_4 + count_5)
    min_count = min([count_1, count_2, count_3, count_4, count_5])
    
    counts = [min_count, min_count, min_count, min_count, min_count]
    for item in game_data_list:
        if counts[int(float(item.consensus))-1] > 0:
            evened_list.append(item)
            counts[int(float(item.consensus))-1] -= 1
            
    print(min_count * 5)
    print(len(evened_list))
    return evened_list

# Evening out the classes doesn't improve anything
# cleaned_data_list = make_even_data(cleaned_data_list)
# random.shuffle(cleaned_data_list)

44750
44750


In [6]:
# Run this
im_cap_concat_list = [np.concatenate((obj.image_embedding, obj.caption_embedding), axis=1)[0].tolist() for obj in cleaned_data_list]
im_cap_concat = np.array(im_cap_concat_list)

# Note that the consensus is rounded, then subtract 1 so that the scores are 0 - 4
labels_list = [int(round(float(obj.consensus))) - 1 for obj in cleaned_data_list]


labels = np.array(labels_list)

In [141]:
# Don't run this cell unless you want to combine classes. 
# Trying to combine into fewer classes. Maybe we just can't distinguish between so many?
# Try 1, 2+3, 4, 5 (4 classes, with 2 and 3 combined)
im_cap_concat_list = [np.concatenate((obj.image_embedding, obj.caption_embedding), axis=1)[0].tolist() for obj in cleaned_data_list]
im_cap_concat = np.array(im_cap_concat_list)

all_labels = [int(round(float(obj.consensus))) for obj in cleaned_data_list]
reduced_all_labels = []
for num in all_labels:
    if num == 1:
        reduced_all_labels.append(num)
    elif num == 2 or num == 3:
        reduced_all_labels.append(2)
    elif num == 4 or num == 5:
        reduced_all_labels.append(3)
    #elif num == 5:
     #   reduced_all_labels.append(4)
    

print('length all labels = ', len(all_labels))
print('length reduced all labels = ', len(reduced_all_labels))
labels_list = [num - 1 for num in reduced_all_labels]

labels = np.array(labels_list)

length all labels =  71078
length reduced all labels =  71078


In [7]:
print(len(im_cap_concat))
print(len(labels))
print('Count 1 = ', labels_list.count(0))
print('Count 2 = ', labels_list.count(1))
print('Count 3 = ', labels_list.count(2))
print('Count 4 = ', labels_list.count(3))
print('Count 5 = ', labels_list.count(4))
print('Total = ', labels_list.count(0) \
      + labels_list.count(1) \
      + labels_list.count(2) \
      + labels_list.count(3) \
      + labels_list.count(4))

35539
35539
Count 1 =  8070
Count 2 =  4475
Count 3 =  7267
Count 4 =  9231
Count 5 =  6496
Total =  35539


In [9]:
print(len(im_cap_concat))
print(len(labels))
print(len(im_cap_concat[0]))
print(im_cap_concat.shape)
print(im_cap_concat[0].shape)
print(im_cap_concat[0])

12039
12039
2048
(12039, 2048)
(2048,)
[-0.03235045 -0.00226173 -0.00101897 ...  0.00793027  0.01999547
  0.04268228]


In [15]:
# Build the model
early_stopping_monitor = EarlyStopping(patience=3)
batch_size = 100
num_epochs = 1000
learning_rate = 0.00001
decay_rate = learning_rate / num_epochs
num_nodes = 1024

model_2 = Sequential()
model_2.add(Dense(num_nodes, activation='relu', input_shape=(2048,)))
model_2.add(Dropout(0.5))
model_2.add(Dense(num_nodes, activation='relu'))
model_2.add(Dropout(0.8))
model_2.add(Dense(num_nodes, activation='relu'))
model_2.add(Dropout(0.5))
# DON'T FORGET TO CHANGE NUMBER OF CLASSES HERE IF CLASSES HAVE BEEN COMBINED!!!
model_2.add(Dense(5, activation='softmax'))
opt = Adam(learning_rate=learning_rate, decay=decay_rate)
loss_func = SparseCategoricalCrossentropy()
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
# NOTE: to use categorical cross entropy, I had to subtract 1 from the consensus to make the ratings from 0 to 4
model_2.compile(optimizer=opt, loss=loss_func, metrics=['accuracy'])
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 5125      
Total params: 2,103,301
Trainable params: 2,103,301
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Training the model
train_size = math.floor(len(im_cap_concat) * 0.8)
print(train_size)

test_size = len(im_cap_concat) - train_size
print(test_size)

model_2.fit(im_cap_concat[:train_size], labels[:train_size], batch_size=batch_size, epochs=num_epochs, \
            validation_split=0.2, callbacks=[tensorboard])



28431
7108
Epoch 1/1000
228/228 [==============================] - 1s 6ms/step - loss: 1.5865 - accuracy: 0.2586 - val_loss: 1.5758 - val_accuracy: 0.2516
Epoch 2/1000
228/228 [==============================] - 1s 5ms/step - loss: 1.5658 - accuracy: 0.2696 - val_loss: 1.5623 - val_accuracy: 0.2553
Epoch 3/1000
228/228 [==============================] - 1s 5ms/step - loss: 1.5521 - accuracy: 0.2779 - val_loss: 1.5493 - val_accuracy: 0.2736
Epoch 4/1000
228/228 [==============================] - 1s 5ms/step - loss: 1.5376 - accuracy: 0.2945 - val_loss: 1.5362 - val_accuracy: 0.2896
Epoch 5/1000
228/228 [==============================] - 1s 5ms/step - loss: 1.5237 - accuracy: 0.3146 - val_loss: 1.5233 - val_accuracy: 0.3030
Epoch 6/1000
228/228 [==============================] - 1s 5ms/step - loss: 1.5111 - accuracy: 0.3255 - val_loss: 1.5108 - val_accuracy: 0.3167
Epoch 7/1000
228/228 [==============================] - 1s 5ms/step - loss: 1.4990 - accuracy: 0.3382 - val_loss: 1.4994 - va

In [17]:
# Predicting and finding accuracy
predictions = model_2.predict(im_cap_concat[train_size:])
test_loss, test_accuracy = model_2.evaluate(im_cap_concat[train_size:], labels[train_size:])
train_loss, train_accuracy = model_2.evaluate(im_cap_concat[:train_size], labels[:train_size]

In [20]:
# Run this
print(predictions[0])

predicted_ratings = []
for pred in predictions:
    predicted_ratings.append(np.argmax(pred)+1)

print(len(predicted_ratings))

# Need to add back the 1 that was subtracted for training
actual_ratings = [int(num) + 1 for num in labels[train_size:]]
print(len(actual_ratings))

[1.8072355e-01 1.0826350e-04 6.5911235e-04 8.7164864e-02 7.3134422e-01]
7108
7108


In [21]:
# Accuracy across classes
def find_accuracy(predicted, actual):
    totals_by_rating = [0, 0, 0, 0, 0, 0]
    correct_by_rating = [0, 0, 0, 0, 0, 0]
    num_correct = 0
    num_ratings = 0
    
    for pred, act in zip(predicted, actual):
        totals_by_rating[int(act)] += 1
        if pred == act:
            num_correct += 1
            correct_by_rating[int(act)] += 1
        num_ratings += 1
        
    print('Accuracy = ', num_correct / num_ratings)
    
    print('---------------------------')
    print('Accuracy by rating 1 - 5')
    print('Test set size=', sum(totals_by_rating))
    print('Number Correct=', sum(correct_by_rating))
    for idx, (correct, total) in enumerate(zip(correct_by_rating, totals_by_rating)):
        if idx == 0:
            continue
        if total == 0:
            continue
        print('Rating = ', idx, '| Accuracy = ', correct/total)
    
    
find_accuracy(predicted_ratings, actual_ratings)

Accuracy =  0.5440348902644907
---------------------------
Accuracy by rating 1 - 5
Test set size= 7108
Number Correct= 3867
Rating =  1 | Accuracy =  0.6405269761606023
Rating =  2 | Accuracy =  0.5508379888268157
Rating =  3 | Accuracy =  0.4340983606557377
Rating =  4 | Accuracy =  0.608410704533042
Rating =  5 | Accuracy =  0.4568487727632621


In [ ]:
print(predicted_ratings)

Using keras tuner for hyperaparameter tuning

In [ ]:
#!pip install -U keras-tuner

In [ ]:
import kerastuner as kt

def build_model(hp):
    inputs = tf.keras.Input(shape=(2048,))
    x = inputs
    for _ in range(3):
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout', 0, 0.3, step=0.05, default=0.2))(x)
    
    outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 0.00001, 0.00002, step=0.000005)),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
    return model
                
                
#tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=300, hyperband_iterations=2, directory='augmented_3layer')

tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=50, directory='random_search_sept_20')

tuner.search(im_cap_concat[:train_size], labels[:train_size], epochs=300, validation_split=0.2, callbacks=[EarlyStopping('val_loss', patience=3)])


In [ ]:
best_model = tuner.get_best_models(1)[0]

In [ ]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print(best_hyperparameters.values)

In [ ]:
tuner.get_best_models()[0].summary()

In [ ]:
tuner.results_summary()